# 🚀 EnfoadsIA Backend - Google Colab (GPU T4)

Este notebook configura y ejecuta el servidor backend de **EnfoadsIA** en Google Colab con GPU T4.

## 📋 Requisitos
- **GPU T4**: Runtime → Change runtime type → GPU
- **Token de Ngrok**: Obtén uno gratis en [ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken)

## 🎯 Instrucciones
1. Ejecuta la **Celda 1**: Instala dependencias (5-10 min)
2. Ejecuta la **Celda 2**: Configura tu token de Ngrok
3. Ejecuta la **Celda 3**: Inicia el servidor
4. **Copia la URL** que aparece (ej: `https://xxxx.ngrok-free.app`)
5. **Pégala en Settings** de tu aplicación frontend

---

In [ ]:
# @title 1️⃣ Configuración Inicial y Dependencias
# @markdown **Ejecuta esta celda primero.** Clona el repositorio e instala todas las dependencias necesarias.

import os
import sys

print("="*60)
print("🚀 ENFOADS IA - CONFIGURACIÓN INICIAL")
print("="*60)

# --- 1.1 Verificar GPU ---
print("\n[1/5] Verificando GPU...")
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

# --- 1.2 Clonar Repositorio ---
REPO_URL = "https://github.com/Juanpalojime/FoadsIA.git"
BRANCH = "master"  # Cambia a 'main' si es necesario

print("\n[2/5] Clonando repositorio...")
if not os.path.exists("FoadsIA"):
    !git clone -b $BRANCH $REPO_URL
    print("✅ Repositorio clonado exitosamente")
else:
    print("⚠️  Repositorio ya existe. Actualizando...")
    !cd FoadsIA && git pull

# --- 1.3 Cambiar al directorio del backend ---
if os.path.exists("FoadsIA/backend"):
    os.chdir("FoadsIA/backend")
    print(f"✅ Directorio de trabajo: {os.getcwd()}")
else:
    print("❌ ERROR: No se encuentra la carpeta backend")
    sys.exit(1)

# --- 1.4 Instalar FFmpeg (necesario para MoviePy) ---
print("\n[3/5] Instalando FFmpeg...")
!apt-get update -qq && apt-get install -y -qq ffmpeg > /dev/null 2>&1
print("✅ FFmpeg instalado")

# --- 1.5 Instalar dependencias de Python ---
print("\n[4/5] Instalando dependencias de Python...")
print("⏳ Esto puede tardar 5-10 minutos. Por favor espera...")
!pip install -q -r requirements.txt
!pip install -q flask-socketio pyngrok eventlet faster-whisper moviepy
print("✅ Todas las dependencias instaladas")

# --- 1.6 Verificar instalación ---
print("\n[5/5] Verificando instalación...")
try:
    import torch
    import flask
    import flask_socketio
    import pyngrok
    print(f"✅ PyTorch: {torch.__version__}")
    print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Flask: {flask.__version__}")
    print(f"✅ Flask-SocketIO instalado")
    print(f"✅ Pyngrok instalado")
except ImportError as e:
    print(f"❌ Error de importación: {e}")

print("\n" + "="*60)
print("✅ CONFIGURACIÓN COMPLETADA")
print("="*60)
print("\n👉 Ahora ejecuta la Celda 2 para configurar Ngrok")

In [ ]:
# @title 2️⃣ Configurar Token de Ngrok
# @markdown Ingresa tu **Authtoken** de Ngrok. Obténlo gratis en: [dashboard.ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken)

NGROK_TOKEN = ""  # @param {type:"string"}

print("="*60)
print("🔐 CONFIGURACIÓN DE NGROK")
print("="*60)

if not NGROK_TOKEN or NGROK_TOKEN.strip() == "":
    print("\n⚠️  ADVERTENCIA: No se proporcionó token de Ngrok")
    print("\n📌 Consecuencias:")
    print("   • La conexión puede caducar en 2 horas")
    print("   • Límite de conexiones simultáneas")
    print("   • URL puede cambiar al reiniciar")
    print("\n💡 Recomendación: Obtén un token gratis en:")
    print("   https://dashboard.ngrok.com/get-started/your-authtoken")
    print("\n⏭️  Puedes continuar sin token, pero es mejor configurarlo.")
else:
    try:
        from pyngrok import conf
        conf.get_default().auth_token = NGROK_TOKEN.strip()
        print("\n✅ Token de Ngrok configurado correctamente")
        print("\n📌 Beneficios:")
        print("   • Conexión estable sin límite de tiempo")
        print("   • Más conexiones simultáneas")
        print("   • URL persistente (opcional)")
    except Exception as e:
        print(f"\n❌ Error al configurar token: {e}")
        print("\n🔧 Intentando configuración alternativa...")
        !ngrok config add-authtoken $NGROK_TOKEN

print("\n" + "="*60)
print("✅ CONFIGURACIÓN DE NGROK COMPLETADA")
print("="*60)
print("\n👉 Ahora ejecuta la Celda 3 para iniciar el servidor")

In [ ]:
# @title 3️⃣ Iniciar Servidor (Auto-Reparable)
# @markdown **Ejecuta esta celda para iniciar el servidor.** La URL de Ngrok aparecerá en la salida.

import os
import sys

print("="*60)
print("🚀 INICIANDO SERVIDOR ENFOADS IA")
print("="*60)

# --- Lógica de Recuperación Automática ---
print("\n[1/3] Verificando archivos...")
if not os.path.exists("FoadsIA") and not os.path.exists("app.py") and not os.path.exists("backend"):
    print("⚠️  No se detectaron archivos. Clonando repositorio...")
    !git clone https://github.com/Juanpalojime/FoadsIA.git
    print("✅ Repositorio clonado")

# --- Búsqueda Inteligente de app.py ---
possible_paths = [
    "app.py",
    "FoadsIA/backend/app.py",
    "backend/app.py"
]

target_dir = None
for path in possible_paths:
    if os.path.exists(path):
        target_dir = os.path.dirname(path) if os.path.dirname(path) else "."
        break

if not target_dir:
    print("\n❌ ERROR FATAL: No se encuentra app.py")
    print(f"\n📂 Contenido del directorio actual:")
    !ls -la
    print("\n💡 Solución: Ejecuta la Celda 1 primero para clonar el repositorio")
    sys.exit(1)

os.chdir(target_dir)
print(f"✅ Ubicación: {os.getcwd()}")

# --- Verificar dependencias críticas ---
print("\n[2/3] Verificando dependencias...")
try:
    import flask_socketio
    import pyngrok
    print("✅ Dependencias verificadas")
except ImportError:
    print("⚠️  Instalando dependencias faltantes...")
    !pip install -q flask-socketio pyngrok eventlet faster-whisper moviepy flask-cors
    print("✅ Dependencias instaladas")

# --- Iniciar Servidor ---
print("\n[3/3] Iniciando servidor Flask + SocketIO...")
print("\n" + "="*60)
print("⚡ SERVIDOR EN EJECUCIÓN")
print("="*60)
print("\n📌 INSTRUCCIONES:")
print("   1. Busca la línea que dice: * Ngrok: https://xxxx.ngrok-free.app")
print("   2. COPIA esa URL completa")
print("   3. Ve a tu aplicación frontend → Settings")
print("   4. Pega la URL y haz clic en 'Guardar y Probar'")
print("\n⚠️  IMPORTANTE: No cierres esta celda mientras uses la aplicación")
print("\n" + "="*60 + "\n")

!python app.py

---

## 🔧 Solución de Problemas

### ❌ Error: "No module named 'flask_socketio'"
**Solución**: Ejecuta la Celda 1 completa y espera a que termine.

### ❌ Error: "No se encuentra app.py"
**Solución**: Verifica que la Celda 1 se ejecutó correctamente. Si no, ejecuta:
```python
!git clone https://github.com/Juanpalojime/FoadsIA.git
%cd FoadsIA/backend
```

### ❌ Error: "CUDA out of memory"
**Solución**: Reinicia el runtime (Runtime → Restart runtime) y vuelve a ejecutar.

### ⚠️ La URL de Ngrok no aparece
**Solución**: Verifica que configuraste el token en la Celda 2.

### 🔄 Reiniciar el servidor
1. Detén la Celda 3 (botón Stop)
2. Vuelve a ejecutarla

---

## 📊 Monitoreo de GPU

Ejecuta esta celda en cualquier momento para ver el uso de GPU:

In [ ]:
# @title 📊 Monitorear GPU
!nvidia-smi

---

## 📚 Recursos Adicionales

- **Repositorio**: [github.com/Juanpalojime/FoadsIA](https://github.com/Juanpalojime/FoadsIA)
- **Ngrok Dashboard**: [dashboard.ngrok.com](https://dashboard.ngrok.com)
- **Documentación**: Ver README.md en el repositorio

---

**Desarrollado con ❤️ usando Flask, PyTorch y modelos de IA de última generación**